# Test align

In [8]:
from koselleck import *

In [9]:
words=get_valid_words()#force=True,only_pos={'nn1'},max_rank=25000)#[:1000]
len(words)

6048

In [10]:
pathdf=get_pathdf_models()
pathdf

,corpus,period_start,period_end,path,path_vocab,run,period
16,bpo,1700,1770,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_01,1700-1770
23,bpo,1700,1770,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_02,1700-1770
5,bpo,1700,1770,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_03,1700-1770
14,bpo,1700,1770,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_04,1700-1770
18,bpo,1700,1770,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_05,1700-1770
...,...,...,...,...,...,...,...
36,bpo,1830,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_08,1830-1900
25,bpo,1830,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_09,1830-1900
34,bpo,1830,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_10,1830-1900
28,bpo,1830,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_11,1830-1900


In [11]:
m1=load_model_row(pathdf.query('period=="1700-1770"').iloc[0])

In [12]:
m2=load_model_row(pathdf.query('period=="1830-1900"').iloc[0])

In [13]:
#dfchange = measure_change(m1,m2,words=words,num_proc=1)

In [14]:
def measure_change_across_runs(
        pathdf = None,
        group_key='period',
        group1_name=None,
        group2_name=None,
        run_key='run',
        num_proc=1,
        words=None,
        num_runs=10,
        **attrs):
    if pathdf is None: pathdf=get_pathdf_models()
    pathdf = pathdf[~pathdf[group_key].isnull()]
    group_names=sorted(list(set(pathdf[group_key])))
    if len(group_names)<2: return pd.DataFrame()
    
    if not group1_name: group1_name=group_names[0]
    if not group2_name: group2_name=group_names[-1]
        
    pathdf_f = pathdf[pathdf[group_key].isin({group1_name,group2_name})]
    pathdf_f = pd.concat([
        grp
        for gi,grp in pathdf_f.groupby(run_key)
        if len(grp)==2
    ][:num_runs])
    
    dfruns = pmap_groups(
        do_measure_changes,
        pathdf_f.groupby(run_key),
        num_proc=num_proc,
        kwargs=dict(
            words=words,
            group_key=group_key,
            progress=1
        ),
        progress=1,
        desc=f'Measuring change across {pathdf_f[run_key].nunique()} period/run model pairs'
    )
    return dfruns

def do_measure_changes(rundf,group_key='period',words=None,**attrs):
    rundf=rundf.sort_values(group_key)
#     display(rundf)
    m1=load_model_row(rundf.iloc[0])
    m2=load_model_row(rundf.iloc[1])
    odf=measure_change(m1,m2,words=words,**attrs).reset_index()
    return odf

In [15]:
dfruns=measure_change_across_runs(
    pathdf,#[pathdf.run.isin({'run_01','run_02','run_03','run_04'})],
    words=words,
    num_proc=5,
    num_runs=10
)
dfruns

Measuring change in abstractness: 100%|██████████| 4764/4764 [00:00<00:00, 10449.58it/s]

Aligning noise aware matrices: iter: 2; alpha: 0.897; sigma: 0.43; sigmay: 1.124: : 4it [00:04,  1.10s/it] 
Aligning noise aware matrices: iter: 5; alpha: 0.894; sigma: 0.426; sigmay: 1.186: : 6it [00:05,  1.06it/s]
Aligning noise aware matrices: iter: 7; alpha: 0.894; sigma: 0.427; sigmay: 1.182: : 8it [00:07,  1.07it/s]
Aligning noise aware matrices: iter: 8; alpha: 0.893; sigma: 0.426; sigmay: 1.208: : 9it [00:08,  1.07it/s]
Aligning noise aware matrices: iter: 8; alpha: 0.892; sigma: 0.425; sigmay: 1.191: : 9it [00:09,  1.04s/it]
Measuring change in abstractness: 100%|██████████| 4766/4766 [00:00<00:00, 8062.82it/s]
Aligning noise aware matrices: iter: 5; alpha: 0.891; sigma: 0.425; sigmay: 1.188: : 6it [00:07,  1.24s/it]
Aligning noise aware matrices: iter: 8; alpha: 0.892; sigma: 0.424; sigmay: 1.209: : 9it [00:12,  1.36s/it]
Aligning noise aware matrices: iter: 7; alpha: 0.893; sigma: 0.427

,word,rank,perc,dist,neighborhood1_local,neighborhood2_local,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,...,dist_noiseaware,dist_procrustes,dist_local,score1_abstractness,score2_abstractness,contrast_abstractness,source_abstractness,period_abstractness,score_diff_abstractness,is_clean_noiseaware
run,,,,,,,,,,,,,,,,,,,,,
run_01,muff,1.50,99.989500,6.974712,"-mutt, -mud, -mould, -stall, -hall, -suffering...","+frock, +wig, +rag, +bonnet, +jacket, +glove, ...",2.0,1.0,1.0,2.0,...,4.867367,5.309358,9.718165,1.420615,-1.294797,Abs-Conc,Median,orig,-2.715412,False
run_01,frill,5.50,99.905502,5.999099,"-rill, -dill, -titan, -woe, -theme, -toil, -mo...","+collar, +jacket, +velvet, +fringe, +ribbon, +...",5.0,6.0,3.0,8.0,...,4.174580,4.745957,7.727703,0.736560,-1.783043,Abs-Conc,Median,orig,-2.519604,True
run_01,slate,9.75,99.816254,6.191828,"-state, -situation, -establishment, -condition...","+chalk, +granite, +grit, +clay, +mortar, +laye...",1.0,21.0,10.0,7.0,...,3.736659,4.369123,8.108449,1.011704,-1.867667,Abs-Conc,Median,orig,-2.879371,False
run_01,stile,9.75,99.816254,5.818356,"-style, -diction, -elegance, -expression, -ima...","+hedge, +glen, +cove, +meadow, +lawn, +arbour,...",4.0,18.0,7.0,10.0,...,3.774124,4.431346,7.444196,0.855246,-1.746647,Abs-Conc,Median,orig,-2.601893,False
run_01,lack,10.00,99.811004,6.067971,"-pig, -hog, -bum, -bitch, -gelding, -team, -wh...","+deficiency, +display, +discernment, +dearth, ...",3.0,15.0,17.0,5.0,...,3.827269,4.190090,8.502950,-1.587326,1.052731,Abs-Conc,Median,orig,2.640057,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
run_10,vehemence,4594.25,3.624633,-1.096487,"calmness, indignation, earnestness, boldness, ...","impetuosity, calmness, impatience, levity, ear...",4372.0,4557.0,4741.0,4707.0,...,-1.246006,-1.451337,-0.711991,1.165348,1.201664,Abs-Conc,Median,orig,0.036316,True
run_10,translation,4610.25,3.288922,-1.165862,"version, translator, text, -fragment, manuscri...","version, translator, text, lexicon, +codex, co...",4356.0,4757.0,4750.0,4578.0,...,-1.557572,-1.480384,-0.652062,0.656403,0.619136,Abs-Conc,Median,orig,-0.037267,True
run_10,christianity,4626.00,2.958456,-1.089289,"religion, revelation, doctrine, paganism, gosp...","religion, paganism, doctrine, atheism, infidel...",4670.0,4659.0,4688.0,4487.0,...,-1.337820,-1.329994,-0.628888,1.749483,1.760774,Abs-Conc,Median,orig,0.011291,True


In [19]:
dfruns.to_csv('data/data.measured_change.runs.v2.csv')

In [20]:
def avg_change_across_runs(dfruns,num_proc=1):
    return pmap_groups(
        do_avg_change_across_runs,
        dfruns.groupby('word'),
        num_proc=num_proc,
        desc='Averaging data across runs'
    ).sort_values('rank')


def do_avg_change_across_runs(dfgrp):
    dfgrp_avg=dfgrp.mean()
    
    # neighbs combine
    def count_neighbs(neighbs):
        return Counter([
            w[1:] if not w[0].isalpha() else w
            for neighb in neighbs
            for w in str(neighb).split(', ')
            if w
        ])
    def reformat_neighbs(neighbs1,neighbs2):
        c1=count_neighbs(neighbs1)
        c2=count_neighbs(neighbs2)
        def do(c1,c2,x='-'):
            return ', '.join([
                f"{(x if not w in c2 else '')}{w} ({c})"
                for w,c in c1.most_common()
                if c>1
            ])
        return do(c1,c2,'-'), do(c2,c1,'+')
    
    neighbs1=dfgrp.neighborhood1_local
    neighbs2=dfgrp.neighborhood2_local
    dfgrp_avg['neighborhood1_local'],dfgrp_avg['neighborhood2_local']=reformat_neighbs(neighbs1,neighbs2)
    dfgrp_avg['score_diff_t_abstractness'],dfgrp_avg['score_diff_p_abstractness']=ttest_ind(
        dfgrp.score1_abstractness,
        dfgrp.score2_abstractness,
    )
        
    return pd.DataFrame([dfgrp_avg])


In [22]:
dfruns=pd.read_csv('data/data.measured_change.runs.v2.csv').set_index('word')#.drop('Unnamed: 0',1)
dfruns_avg=avg_change_across_runs(dfruns,num_proc=7)

Averaging data across runs [x7]: 100%|██████████| 4824/4824 [00:06<00:00, 798.39it/s]


In [23]:
dfruns_avg

,rank,perc,dist,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,perc_abstractness,perc_noiseaware,perc_procrustes,...,dist_procrustes,dist_local,score1_abstractness,score2_abstractness,score_diff_abstractness,is_clean_noiseaware,neighborhood1_local,neighborhood2_local,score_diff_t_abstractness,score_diff_p_abstractness
word,,,,,,,,,,,,,,,,,,,,,
muff,2.275,99.973224,6.790697,2.1,2.0,2.6,2.4,99.976891,99.978998,99.966393,...,5.020819,9.399048,1.401059,-1.333100,-2.734159,0.0,"-mutt (10), -mud (10), -mould (10), -stall (10...","+frock (10), +wig (10), +bonnet (10), +glove (...",46.864062,2.886021e-20
slate,6.000,99.895008,6.324146,1.5,10.1,5.9,6.5,99.989504,99.808923,99.897106,...,4.649457,8.242779,1.076886,-1.741295,-2.818181,0.0,"-state (10), -situation (10), -establishment (...","+chalk (10), +granite (10), +grit (10), +clay ...",81.890392,1.310934e-24
stile,10.400,99.802606,5.681941,6.4,14.5,8.9,11.8,99.886592,99.716534,99.834111,...,4.487370,7.417718,0.819356,-1.575376,-2.394732,0.0,"-style (10), -diction (10), -elegance (10), -e...","+hedge (10), +glen (10), +meadow (10), +lawn (...",58.285564,5.834543e-22
frill,12.900,99.750095,5.577451,6.2,12.0,12.5,20.9,99.890818,99.768988,99.758474,...,4.446066,6.700414,0.697979,-1.744639,-2.442619,1.0,"-rill (10), -dill (10), -woe (10), -theme (10)...","+collar (10), +velvet (10), +fringe (10), +rib...",29.727429,9.419315e-17
tact,15.275,99.700192,5.471668,16.7,22.2,18.4,3.8,99.670299,99.554732,99.634540,...,4.169248,8.786855,-0.752793,1.124971,1.877764,0.0,"-mic (10), -tau (9), -vim (9), -tit (8), -lien...","+ability (10), +dexterity (10), +skill (10), +...",-39.247236,6.823752e-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
love,4517.800,5.147262,-1.106743,4356.6,4734.2,4694.5,4285.9,8.532553,0.602774,1.436815,...,-1.372177,-0.595129,0.950635,0.948341,-0.002294,1.0,"lover (10), passion (10), pity (10), rapture (...","affection (10), passion (10), tenderness (10),...",0.159039,8.754091e-01
grief,4527.500,4.943678,-1.075415,4198.6,4653.2,4701.9,4556.3,11.852832,2.303467,1.280757,...,-1.377595,-0.665919,0.641351,0.657974,0.016623,1.0,"sorrow (10), anguish (10), sigh (10), woe (10)...","sorrow (10), anguish (10), remorse (10), sadne...",-0.830267,4.172642e-01
truth,4531.425,4.861039,-1.027810,4477.0,4583.4,4481.0,4584.3,6.003998,3.769834,5.919803,...,-1.156314,-0.671969,1.756064,1.764733,0.008670,1.0,"falsehood (10), sincerity (10), fallacy (10), ...","revelation (10), falsehood (10), sincerity (10...",-0.718520,4.816602e-01


In [24]:
dfruns_avg.to_csv('data/data.measured_change.runs_avg.v2.csv')

In [25]:
dfruns_avg.query('rank<1000').sample(n=10).sort_values('rank_local')

,rank,perc,dist,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,perc_abstractness,perc_noiseaware,perc_procrustes,...,dist_procrustes,dist_local,score1_abstractness,score2_abstractness,score_diff_abstractness,is_clean_noiseaware,neighborhood1_local,neighborhood2_local,score_diff_t_abstractness,score_diff_p_abstractness
word,,,,,,,,,,,,,,,,,,,,,
flue,72.075,98.507342,3.439673,134.2,46.8,39.7,67.6,97.202290,99.038317,99.187371,...,3.690162,4.220748,-0.489971,-1.566804,-1.076833,1.0,"-tier (10), -aunt (10), -mamma (10), -husband ...","+boiler (10), +tube (10), +stove (10), +roller...",25.975562,1.013426e-15
longing,227.325,95.247039,2.252841,34.7,337.0,269.8,267.8,99.292216,92.944057,94.355086,...,1.827687,1.493726,-0.778116,0.758560,1.536676,0.7,"weeping (10), -transport (10), -twain (10), -s...","+craving (10), +sadness (10), gladness (10), +...",-38.428168,9.932184e-19
stew,338.875,92.903412,1.875246,810.5,139.1,136.5,269.4,82.997423,97.099366,97.153893,...,2.572145,1.483710,-0.835427,-1.459336,-0.623909,1.0,"-sob (10), -sty (9), -batt (8), -solo (7), -ai...","+broth (10), +boil (10), +vinegar (10), +butte...",9.236144,2.990050e-08
organ,491.450,89.700287,1.262360,917.7,322.9,434.2,291.0,80.748390,93.240090,90.903130,...,1.374565,1.343859,-0.570569,-0.037819,0.532750,0.0,"flute (10), music (10), violin (10), -machine ...","instrument (10), +organist (10), flute (10), c...",-18.569397,3.454634e-13
kilt,355.525,92.555909,1.603052,269.5,275.8,285.7,591.1,94.362587,94.229649,94.021797,...,1.787707,0.693783,-0.593662,-1.548849,-0.955187,1.0,"mic (9), sim (9), lag (8), -lien (7), hilt (7)...","+wig (10), +plaid (10), +frock (10), kid (10),...",13.362276,8.773110e-11
twill,686.850,85.597437,1.089945,1395.4,309.0,348.5,694.5,70.717852,93.531785,92.702332,...,1.615077,0.491750,-0.578160,-1.069267,-0.491107,1.0,"-stall (10), -bid (10), -wail (10), trill (10)...","+vat (10), +bra (9), +tab (9), tau (8), +pap (...",6.620117,3.246154e-06
lather,626.400,86.867563,1.120151,392.6,593.5,682.7,836.8,91.777451,87.557783,85.685401,...,1.062699,0.366500,-0.036897,-0.913208,-0.876311,1.0,"-grandmother (10), -filter (10), -uncle (10), ...","wile (10), hoe (10), sou (9), +pate (9), +lard...",10.458836,4.458243e-09
mood,656.075,86.243551,1.256078,88.0,850.5,776.7,909.1,98.173058,82.160809,83.711156,...,0.863578,0.267887,-0.462448,0.776559,1.239007,0.8,"strain (10), vein (10), -balm (10), -moan (10)...","sadness (10), +reverie (10), vein (10), +tempe...",-19.648207,1.308219e-13
gathering,944.975,80.175853,0.609728,1240.3,866.7,715.4,957.5,73.973224,81.820388,84.997323,...,0.921776,0.244437,-0.870094,-0.408759,0.461335,1.0,"feeding (10), -moisture (10), grass (10), spre...","+throng (10), spreading (10), +concourse (10),...",-13.814092,5.071703e-11


In [26]:
dfruns_avg.loc['value']

rank                                                                   2156.65
perc                                                                   54.7315
dist                                                                  0.074118
rank_abstractness                                                        278.9
rank_noiseaware                                                         3767.8
rank_procrustes                                                         3127.1
rank_local                                                              1452.8
perc_abstractness                                                      94.1638
perc_noiseaware                                                        20.8976
perc_procrustes                                                        34.3517
perc_local                                                             69.5129
dist_abstractness                                                      1.71869
dist_noiseaware                                     